In [85]:
from binance.client import Client
from binance.websockets import BinanceSocketManager
import pandas as pd
from datetime import datetime
import time

In [133]:
pd.set_option('display.precision',8)

In [63]:
api_key = 'H99oizz3xqSP9vDf0CFnKOfqiudL6XDnqum04OZbmUOG0wNDV8e9yjsvsbYmy6oR'
api_secret = 'JEWZAnuNybCWfPVx6WFD5FpOB9vZgD2DOSYw1sk3swfFFy3b82arbJFL0DmuQChc'
client = Client(api_key, api_secret)

In [80]:
def daily_price_excel(klines,symbol):
    df_list = []
    for i in range(0,len(klines)):
        daily_data = {}
        daily_data['time'] = datetime.fromtimestamp(klines[i][0]/1000)
        daily_data['open'] = klines[i][1]
        daily_data['high'] = klines[i][2]
        daily_data['low'] = klines[i][3]
        daily_data['close'] = klines[i][4]
        df_list.append(pd.DataFrame(daily_data,index = [0]))
    daily_price = pd.concat(df_list)
    daily_price['ticker'] = symbol
    daily_price= daily_price.reset_index(drop = True)
    #drop this minute because it is unfinished data
    daily_price.drop(daily_price.shape[0]-1,inplace = True)
    name = str(daily_price.loc[daily_price.shape[0]-1,'time'])
    name = name.replace(":",'')
    #daily_price.to_csv(name+".csv",index = False)
    return daily_price

#### 1. get all symbol list from Binance

In [81]:
response = client.get_exchange_info()
symbol_list = [item['symbol'] for item in response['symbols']]

In [106]:
unique_symbol = [symbol for symbol in symbol_list if symbol[-3:] == 'BTC']

#### 2. get 3 day historical prices

In [103]:
def get_historical_prices(symbol_list,time_range = "4 day ago UTC"):
    total_prices = []
    for symbol in symbol_list:
        print(symbol)
        klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_30MINUTE,time_range )
        ##check if empty data or not
        if len(klines)>0:
            single_prices = daily_price_excel(klines, symbol)
            total_prices.append(single_prices)
            time.sleep(0.1)
        else:
            continue
    output_prices = pd.concat(total_prices)
    return output_prices

In [109]:
btc_quote = get_historical_prices(unique_symbol,time_range = "4 day ago UTC")

ETHBTC
LTCBTC
BNBBTC
NEOBTC
BCCBTC
GASBTC
HSRBTC
MCOBTC
WTCBTC
LRCBTC
QTUMBTC
YOYOBTC
OMGBTC
ZRXBTC
STRATBTC
SNGLSBTC
BQXBTC
KNCBTC
FUNBTC
SNMBTC
IOTABTC
LINKBTC
XVGBTC
SALTBTC
MDABTC
MTLBTC
SUBBTC
EOSBTC
SNTBTC
ETCBTC
MTHBTC
ENGBTC
DNTBTC
ZECBTC
BNTBTC
ASTBTC
DASHBTC
OAXBTC
ICNBTC
BTGBTC
EVXBTC
REQBTC
VIBBTC
TRXBTC
POWRBTC
ARKBTC
XRPBTC
MODBTC
ENJBTC
STORJBTC
VENBTC
KMDBTC
RCNBTC
NULSBTC
RDNBTC
XMRBTC
DLTBTC
AMBBTC
BATBTC
BCPTBTC
ARNBTC
GVTBTC
CDTBTC
GXSBTC
POEBTC
QSPBTC
BTSBTC
XZCBTC
LSKBTC
TNTBTC
FUELBTC
MANABTC
BCDBTC
DGDBTC
ADXBTC
ADABTC
PPTBTC
CMTBTC
XLMBTC
CNDBTC
LENDBTC
WABIBTC
TNBBTC
WAVESBTC
GTOBTC
ICXBTC
OSTBTC
ELFBTC
AIONBTC
NEBLBTC
BRDBTC
EDOBTC
WINGSBTC
NAVBTC
LUNBTC
TRIGBTC
APPCBTC
VIBEBTC
RLCBTC
INSBTC
PIVXBTC
IOSTBTC
CHATBTC
STEEMBTC
NANOBTC
VIABTC
BLZBTC
AEBTC
RPXBTC
NCASHBTC
POABTC
ZILBTC
ONTBTC
STORMBTC
XEMBTC
WANBTC
WPRBTC
QLCBTC
SYSBTC
GRSBTC
CLOAKBTC
GNTBTC
LOOMBTC
BCNBTC
REPBTC
TUSDBTC
ZENBTC
SKYBTC
CVCBTC
THETABTC
IOTXBTC
QKCBTC
AGIBTC
NXSBTC
DATABTC
SCBTC
NPXS

#### 3. convert price to USDT quote

In [158]:
def convert_2_USD_price(all_btc_price, pair = 'BTCUSDT', time_range = '4 day ago UTC'):
    usd_price = get_historical_prices([pair],time_range = time_range)
    
    ## change format
    columns = ['open','high','low','close']
    for column in columns:
        all_btc_price[column] = all_btc_price[column].astype('float')
        usd_price[column] = usd_price[column].astype('float')
        
    usd_price.columns = ['time','open_','high_','low_','close_','ticker_']

    all_usd_price = all_btc_price.merge(usd_price, on = 'time')
    all_usd_price['open'] = all_usd_price['open'] * all_usd_price['open_']
    all_usd_price['high'] = all_usd_price['high'] * all_usd_price['high_']
    all_usd_price['low'] = all_usd_price['low'] * all_usd_price['low_']
    all_usd_price['close'] = all_usd_price['close'] * all_usd_price['close_']
    
    #update ticker
    all_usd_price['ticker'] = all_usd_price['ticker'].str[:-3] + all_usd_price['ticker_'].str[3:] 
    all_usd_price = all_usd_price[['time','ticker','open','high','low','close']]
    return all_usd_price
    

In [159]:
usd_quote =convert_2_USD_price(btc_quote, pair = 'BTCUSDT', time_range = '4 day ago UTC')

BTCUSDT


#### 4. get only close price

In [171]:
close_usd_quote = usd_quote[(usd_quote['time'].dt.hour == 00) & (usd_quote['time'].dt.minute == 00)]

In [173]:
close_usd_quote.to_excel('Binance_price_date.xlsx',index = False)